In [1]:
import tushare as ts
import pandas as pd
from influxdb import InfluxDBClient
import datetime
from tqdm import tqdm
import time

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')
pro = ts.pro_api()

In [4]:
data = []
for date in tqdm(pd.date_range(start='2015-01-01', end='2020-05-26')):
    date_str = date.strftime("%Y%m%d")
    df = pro.anns(ann_date=date_str)
    data.append(df)
    time.sleep(2)

100%|██████████| 1973/1973 [1:36:16<00:00,  2.93s/it]


In [17]:
ann_df = pd.concat(data)

In [18]:
ann_df['ann_date'] = pd.to_datetime(ann_df['ann_date'])
ann_df['title'] = ann_df['title'].astype(str)
ann_df['content'] = ann_df['content'].astype(str)

In [19]:
len(ann_df)

79648

In [20]:
ann_df.head()

,ts_code,ann_date,title,content
0,300081.SZ,2015-01-01,关于重大资产重组进展暨延期复牌公告,证券代码：300081 证券简称：恒信移动 ...
1,600789.SH,2015-01-01,2014年度业绩预亏公告,证券代码:600789 股票简称:鲁抗医药 公告...
2,300068.SZ,2015-01-01,关于持股5%以上股东减持股份的提示性公告,\n证券代码：300068 证券简称：南都电源 ...
3,300109.SZ,2015-01-01,关于召开2015年第一次临时股东大会的通知,证券代码：300109 证券简称：新开源 公告编号：201...
4,600458.SH,2015-01-01,第六届监事会第十八次（临时）会议决议公告,\n股票代码：600458 股票简称：时代新材 编号...


In [40]:
client = InfluxDBClient('localhost', 8086, database='stock')

In [41]:
js = []
for i, row in ann_df.iterrows():
    objs = {
        "measurement": 'announcement',
        "tags": {
            'companyId': row['ts_code']
        },
        "time": row['ann_date'],
        "fields": {
            'title': row['title'],
            'content': row['content']
        }
    }
    js.append(objs)

In [42]:
js[0]

{'measurement': 'announcement',
 'tags': {'companyId': '300081.SZ'},
 'time': Timestamp('2015-01-01 00:00:00'),
 'fields': {'title': '关于重大资产重组进展暨延期复牌公告',
  'content': '证券代码：300081            证券简称：恒信移动              公告编号：2014-053 \n \n恒信移动商务股份有限公司 \n关于重大资产重组进展暨延期复牌公告 \n本公司及其董事会全体成员保证公告内容真实、准确和完整，公告不存在虚\n假记载、误导性陈述或重大遗漏 \n \n     恒信移动商务股份有限公司（以下简称“公司”）于2014年12月5日披露了《重\n大事项停牌公告》，公司股票于2014年12月5日开市起停牌。2014年12月19日，\n公司披露了《重大资产重组停牌公告》，公司正在筹划重大资产重组事项。2014\n年12月25日，公司召开第五届董事会第八次会议，审议通过了《关于同意公司筹\n划重大资产重组事项的议案》，同意公司筹划重大资产重组事项，并于当天披露\n了《重大资产重组进展公告》。 \n本公司原预计于2015年1月5日前按照《公开发行证券的公司信息披露内容与\n格式准则第26 号－上市公司重大资产重组申请文件》的要求披露重大资产重组\n预案，自停牌以来，公司、重组有关各方及聘请的中介机构积极推动各项工作，\n目前正在抓紧对涉及重组事项的相关资产进行尽职调查、审计、评估等。截至本\n公告日，公司本次重大资产重组事项的相关准备工作尚未全部完成，重组方案的\n相关内容仍需要进一步商讨、论证和完善。为保护广大投资者权益，公司已向深\n交所申请延长股票停牌时间，公司股票将于2015 年1月5日起继续停牌，预计于\n2015年2月5日前复牌并披露本次重大资产重组预案或报告书。 \n    逾期未能披露发行股份购买资产预案或报告书的，公司将根据重组推进情况\n确定是否向交易所申请延期复牌。公司未提出延期复牌申请或延期复牌申请未获\n同意的，公司股票将于2015 年2月5日恢复交易，并自公司股票复牌之日起六个\n月内不再筹划重大资产重组事项。如公司在停牌期限内终止筹划重大资产重组\n的，公司将及时

In [44]:
len(js)

79648

In [46]:
client.write_points(js, batch_size=100)

True